# openpyxl

https://openpyxl.readthedocs.io/en/stable/

In [33]:
# 读取原数据
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Font
import xlrd
ori_excel = xlrd.open_workbook('1.xlsx')
sheet = ori_excel.sheet_by_index(0)
data = list()
# start_row = 
for i in range(7, sheet.nrows):
    values = sheet.row_values(i)
    d = dict()
    d['位号'] =values[2] 
    d['用途'] =values[3]
    d['数量'] =int(values[4])
    d['管号'] =values[6]
    d['管道尺寸'] =values[9]
    d['管道材质'] =values[10]
    d['PID'] =int(values[5])
    d['流体'] =values[12]
    d['温度正常'] =int(values[15])
    d['温度最高'] =int(values[14])
    d['压力正常'] =int(values[18])
    d['压力最高'] =int(values[17])
    d['密度'] = int(values[19])
    data.append(d)

In [36]:
wb = load_workbook(filename='2.xlsx')
ws = wb[wb.sheetnames[0]]

In [230]:
def set_value(value, ws, position, merge_range):
    ws[position].value = value
    ws.merge_cells(merge_range)
    ws[position].alignment = Alignment(horizontal='center', vertical='center')

# def fill_values(d, ws, save_path):
#     # 位号 Tag Number
#     set_value(d['位号'], ws, position='X8', merge_range='X8:AG8')
#     set_value(d['数量'], ws, position='AH8', merge_range='AH8:AQ8')
#     set_value(d['用途'], ws, position='X9', merge_range='X9:AG9')
#     set_value(d['管号'], ws, position='X10', merge_range='X10:AG10')
#     set_value(d['管道尺寸'], ws, position='X11', merge_range='X11:AG11')
#     set_value(d['管道材质'], ws, position='X13', merge_range='X13:AG13')
#     set_value(d['PID'], ws, position='X14', merge_range='X14:AG14')
#     set_value(d['流体'], ws, position='X16', merge_range='X16:AG16')

#     set_value(d['温度正常'], ws, position='AC18', merge_range='AC18:AG18')
#     set_value(d['温度最高'], ws, position='AH18', merge_range='AH18:AL18')
#     set_value(d['压力正常'], ws, position='AC19', merge_range='AC19:AG19')
#     set_value(d['压力最高'], ws, position='AH19', merge_range='AH19:AL19')
#     set_value(d['密度'], ws, position='AC21', merge_range='AC21:AG21')

#     # 调整字体
#     for row in ws.rows:
#         for cell in row:
#             if not cell.value:
#                 continue
#             params = {
#                 'family': cell.font.family,
#                 'b': cell.font.b,
#                 'sz': cell.font.sz,
#                 'charset': cell.font.charset,
#             }
#             cell.font = Font(name=u'宋体')
#             cell.font = Font(name='Times New Roman', **params)

    # 保存文件
#     wb.save(save_path)
from tqdm.auto import tqdm


# 拷贝区域

In [209]:
import copy
from tqdm.auto import tqdm


def copy_cell(src_cell, dst_cell):
    # print(src_cell.value)
    dst_cell.value = src_cell.value
    dst_cell.border = copy.copy(src_cell.border)
    dst_cell.font = copy.copy(src_cell.font)
    dst_cell._style = copy.copy(src_cell._style)
    dst_cell.fill = copy.copy(src_cell.fill)
    dst_cell.number_format = copy.copy(src_cell.number_format)
    dst_cell.protection = copy.copy(src_cell.protection)
    dst_cell.alignment  = copy.copy(src_cell.alignment)
    

def copy_sheet_range(ws, sheet_area, num):
    merge_ranges = []
    for cell_range in ws.merged_cells.ranges:
        merge_ranges.append(cell_range.bounds)
    
    length = len(sheet_area)
    for i in range(num):
        shift = (length + 1) * (i + 1)
        # print(shift)
        for j, rows in tqdm(enumerate(sheet_area), total=length):
            ws.row_dimensions[j+shift].height = ws.row_dimensions[j].height
            for k, cell in enumerate(rows):
                dst_cell = ws[cell.row + shift][cell.column-1]
                copy_cell(cell, dst_cell)

        for start_column, start_row, end_column, end_row in merge_ranges:
            ws.merge_cells(start_row=start_row+shift, start_column=start_column, end_row=end_row+shift, end_column=end_column)
        
        
def set_cell_font(ws):
        # 调整字体
    for row in tqdm(ws.rows, total=ws.max_row):
        for cell in row:
            if not cell.value:
                continue
            params = {
                'family': cell.font.family,
                'b': cell.font.b,
                'sz': 10,
                'charset': cell.font.charset,
            }
            cell.font = Font(name=u'宋体')
            if type(cell.value) is str and '订货' not in cell.value:
                cell.font = Font(name='Times New Roman', **params)

In [76]:
wb = load_workbook(filename='2.xlsx')
ws = wb[wb.sheetnames[0]]
set_cell_font(ws)
copy_sheet_range(ws, sheet_area=ws['B2:AQ64'], num=2)
wb.save('2325.xlsx')

# PDF识别


In [ ]:
import pdfplumber
import pandas as pd

In [234]:
re.findall(r'-?\d+', 'asdfasdf -59, 77, -23')

['-59', '77', '-23']

In [254]:
import re

def get_table_data(tables):
    d = dict()

    d['位号'] = tables[0][5][1][4:]
    if len(tables) <= 1:
        return d
    
    d['数量'] = tables[1][1][1].replace('Quantity ', '')
    # d['pipe_class'] = tables[1][6][1].replace('Process Medium Spent', '').strip()
    d['管道编号'] = tables[1][8][1].replace('Pipe Class', '').strip()
    d['设备编号'] = tables[1][8][6].replace('Vessel Number', '').strip()

    ans = re.search(r'(DN) \d+',tables[1][10][1])
    d['管道尺寸'] = ans.group(0).replace(' ', '') if ans else ''

    ans = re.search(r'(PN) \d+',tables[1][10][1])
    d['管道等级'] = ans.group(0).replace(' ', '') if ans else ''

    if 'liquid' in tables[1][6][6]:
        state = '液'
    elif 'gaseous' in tables[1][6][6]:
        state = '气'
    else:
        state = '无'
    d['状态'] = state

    d['管道材质'] = tables[1][12][6].replace('Vessel Material', '').strip()
    
    r = re.compile(r'(-?\d+,?\d+)')
    ans = r.search(tables[1][18][4])
    d['温度最小'] = ans.group(1).replace(',', '.') if ans else ''
    ans = r.search(tables[1][18][2])
    d['温度正常'] = ans.group(1).replace(',', '.') if ans else ''
    ans = r.search(tables[1][18][7])
    d['温度最大'] = ans.group(1).replace(',', '.') if ans else ''

    d['温度单位'] = tables[1][18][10].split(',')[0]

    ans = r.search(tables[1][15][4])
    d['压力最小'] = ans.group(1).replace(',', '.') if ans else ''
    ans = r.search(tables[1][15][2])
    d['压力正常'] = ans.group(1).replace(',', '.') if ans else ''
    ans = r.search(tables[1][15][7])
    d['压力最大'] = ans.group(1).replace(',', '.') if ans else ''
    d['压力单位'] = tables[1][15][10].split(',')[0]

    ans = r.search(tables[1][17][4])
    d['差压最小'] = ans.group(1).replace(',', '.') if ans else ''
    ans = r.search(tables[1][17][2])
    d['差压正常'] = ans.group(1).replace(',', '.') if ans else ''
    ans = r.search(tables[1][17][7])
    d['差压最大'] = ans.group(1).replace(',', '.') if ans else ''
    d['差压单位'] = tables[1][17][10].split(',')[0]

    ans = r.search(tables[1][19][4])
    d['密度最小'] = ans.group(1).replace(',', '.') if ans else ''
    ans = r.search(tables[1][19][2])
    d['密度正常'] = ans.group(1).replace(',', '.') if ans else ''
    ans = r.search(tables[1][19][7])
    d['密度最大'] = ans.group(1).replace(',', '.') if ans else ''
    d['密度单位'] = tables[1][19][10].split(',')[0]
    return d

In [247]:
r = re.compile(r'(-?\d+,?\d+)')

In [248]:
r.findall('asdfasdf -1,34 asdf 0,310 asdf')

['-1,34', '0,310']

In [250]:
def set_value(value, ws, position, merge_range):
    ws[position].value = value
    ws.merge_cells(merge_range)
    ws[position].alignment = Alignment(horizontal='center', vertical='center')

def fill_values(d, ws, shift=0):
    # 位号 Tag Number
    set_value(d['位号'], ws, position=f'X{8+shift}', merge_range=f'X{8+shift}:AG{8+shift}')
    
    if '数量' not in d:
        return 
    
    set_value(d['数量'], ws, position=f'AH{8+shift}', merge_range=f'AH{8+shift}:AQ{8+shift}')
    
    set_value(d['管道编号'], ws, position=f'X{10+shift}', merge_range=f'X{10+shift}:AG{10+shift}')
    set_value(d['设备编号'], ws, position=f'AH{10+shift}', merge_range=f'AH{10+shift}:AQ{10+shift}')
    
    set_value(d['管道尺寸'], ws, position=f'X{11+shift}', merge_range=f'X{11+shift}:AG{11+shift}')
    set_value(d['管道等级'], ws, position=f'AH{11+shift}', merge_range=f'AH{11+shift}:AQ{11+shift}')
    
    set_value(d['管道材质'], ws, position=f'X{13+shift}', merge_range=f'X{13+shift}:AG{13+shift}')
    
    set_value(d['状态'], ws, position=f'AH{16+shift}', merge_range=f'AH{16+shift}:AQ{16+shift}')
    
    index = 18 + shift
    set_value(d['温度最小'], ws, position=f'X{index}', merge_range=f'X{index}:AB{index}')
    set_value(d['温度正常'], ws, position=f'AC{index}', merge_range=f'AC{index}:AG{index}')
    set_value(d['温度最大'], ws, position=f'AH{index}', merge_range=f'AH{index}:AL{index}')
    set_value(d['温度单位'], ws, position=f'AM{index}', merge_range=f'AM{index}:AQ{index}')
    index = 19 + shift
    set_value(d['压力最小'], ws, position=f'X{index}', merge_range=f'X{index}:AB{index}')
    set_value(d['压力正常'], ws, position=f'AC{index}', merge_range=f'AC{index}:AG{index}')
    set_value(d['压力最大'], ws, position=f'AH{index}', merge_range=f'AH{index}:AL{index}')
    set_value(d['压力单位'], ws, position=f'AM{index}', merge_range=f'AM{index}:AQ{index}')
    index = 20 + shift
    set_value(d['差压最小'], ws, position=f'X{index}', merge_range=f'X{index}:AB{index}')
    set_value(d['差压正常'], ws, position=f'AC{index}', merge_range=f'AC{index}:AG{index}')
    set_value(d['差压最大'], ws, position=f'AH{index}', merge_range=f'AH{index}:AL{index}')
    set_value(d['差压单位'], ws, position=f'AM{index}', merge_range=f'AM{index}:AQ{index}')
    index = 21 + shift
    set_value(d['密度最小'], ws, position=f'X{index}', merge_range=f'X{index}:AB{index}')
    set_value(d['密度正常'], ws, position=f'AC{index}', merge_range=f'AC{index}:AG{index}')
    set_value(d['密度最大'], ws, position=f'AH{index}', merge_range=f'AH{index}:AL{index}')
    set_value(d['密度单位'], ws, position=f'AM{index}', merge_range=f'AM{index}:AQ{index}')

In [ ]:
# df = pd.DataFrame(table[1:], columns=table[0])
pdf = pdfplumber.open("202386_M669_I-110_Transmitter (Standard)_Rev00.pdf")
p = pdf.pages[3]
tables = p.extract_tables()
print('总共%s个表' % len(tables))

In [251]:
# wb = load_workbook(filename='2.xlsx')
# ws = wb[wb.sheetnames[0]]
# set_cell_font(ws)
# copy_sheet_range(ws, sheet_area=ws['B2:AQ64'], num=74)
# wb.save('step1.xlsx')

In [252]:
wb = load_workbook(filename='step1.xlsx')
ws = wb[wb.sheetnames[0]]
set_cell_font(ws)

In [255]:
for i in tqdm(range(74)):
    shift = i * 64
    p = pdf.pages[3+i]
    tables = p.extract_tables()
    d = get_table_data(tables)
    fill_values(d, ws, shift=shift)
    # print(d)

In [256]:
wb.save('step2.xlsx')